## Setup and configuration

1. Firstly, we'll import usefull packages.
2. Then, we'll load the data, before visualize and preprocess it.
3. We'll try a simple CNN model and then we will evaluate its performances.
4. And finally, we'll use techniques such as data augmentation, learning rate decay and dropout to increase our model's accuracy.

**Import Packages**

In [0]:
import numpy as np                          # linear algebra
import os                                   # used for loading the data
from sklearn.metrics import confusion_matrix# confusion matrix to carry out error analysis
import seaborn as sn                        # heatmap
from sklearn.utils import shuffle           # shuffle the data
import matplotlib.pyplot as plt             # 2D plotting library
import cv2                                  # image processing library
import tensorflow as tf                     # best library ever

**Import datas**

In [0]:
train_url = 
test_url =

In [0]:
from keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator()

#Load dataset
train_generator = train_datagen.flow_from_directory(
        'temp/fruits-360/Training',
        target_size=(100, 100),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'temp/fruits-360/Test',
        target_size=(100, 100),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical')